In [1]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
except:
  # We are not in Google Colab
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

$\large{\text{Foundations of Kinematics and Algorithms in Robotics}} \\ \large{\text{Module 3: Twists and Interpolation}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

### Contents

[1.0 What is Position?](#1)  
[2.0 What is Orientation?](#2)  
[3.0 Homogeneous Coordinates](#3)  
[4.0 Pose Graphs](#4)  

In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm
import spatialmath.base as smb

import spatialgeometry as sg

# The matrix exponential method
from scipy.linalg import expm, logm

# sympy provides symbolic mathematic operations 
import sympy as sym

# setting up sympy to print nicely
from IPython import display as ds
from sympy.physics.mechanics import init_vprinting
from sympy.physics.vector import vlatex
init_vprinting(use_latex='mathjax')

Videos

[What is e?](https://youtu.be/O85OWBJ2ayo)

<br>

<a id='2'></a>
# 1.0 Twists
---

Earlier, we discussed the concept of Lie groups and we explained how we can get an $\mathbf{SO}(3)$ from the angle-axis representation:

Given the angle axis representation where
$$
    \hat{\mathbf{\eta}} = \begin{pmatrix}
        \hat{\eta}_1 \\
        \hat{\eta}_2 \\
        \hat{\eta}_3
    \end{pmatrix}
$$

defines the axis of rotation, and $\theta$ is the angle of rotation, we can get the Euler vector (also known as the exponential coordinates) by

\begin{equation*}
    \bf{\eta} =
    \hat{\eta} \ \theta \in \mathbb{R}^3
\end{equation*}

and we can turn the Euler vector into an $\mathbf{so}(3)$ matrix (the Lie algebra of the $\mathbf{SO}(3)$ group) through the `skew` operation

$$
    [\mathbf{\eta}]_\times =
    \begin{pmatrix}
        0 & -\eta_3 & \eta_2 \\
        \eta_3 & 0 & -\eta_1 \\
        -\eta_2 & \eta_1 & 0
    \end{pmatrix} \in \mathbf{so}(3).
$$

Finally, we can construct the $\mathbf{SO}(3)$ matrix from the $\mathbf{so}(3)$ matrix using the matrix exponential

$$
    \mathbf{R}(\mathbf{\eta}) = e^{[\mathbf{\eta}]_\times} \in \mathbf{SO}(3).
$$

We can also follow this process with a vector $\mathbf{s} \in \mathbb{R}^6$ to get an $\mathbf{SE}(3)$ matrix. Through the vector $\mathbf{s}$, we can define a pose (translation and rotation).

## Representing Pose as a Twist

We can represent pose as a twist

$$
    \mathbf{s} =
    \begin{pmatrix}
        \upsilon_1 \\
        \upsilon_2 \\
        \upsilon_3 \\
        \eta_1 \\
        \eta_2 \\
        \eta_3
    \end{pmatrix}
    \in \mathbb{R}^6
$$

where the first three elements $\mathbf{\upsilon}$ define the translation and the last three elements $\mathbf{\eta}$ define the rotation.

_Note that we are using the Greek letter upsilon $\upsilon$ (not to be confused with the English letter $v$ or the Greek letter nu $\nu$) to represent the translation component._



We can convert a twist into an augmented skew symmetric matrix (the Lie algebra of the $\mathbf{SE}(3)$ group) using the `skewa` operation

$$
    [\mathbf{s}] =
    \begin{pmatrix}
        0 & -\eta_3 & \eta_2 & \upsilon_1 \\
        \eta_3 & 0 & -\eta_1 & \upsilon_2 \\
        -\eta_2 & \eta_1 & 0 & \upsilon_3 \\
        0 & 0 & 0 & 0
    \end{pmatrix} \in \mathbf{se}(3).
$$

and we can convert this matrix into an $\mathbf{SE}(3)$ matrix using the matrix exponential

$$
    \mathbf{T}(\mathbf{s}) = e^{[\mathbf{s}]} \in \mathbf{SE}(3).
$$

## The Zero Twist

What happens if we make a twist with all zeros?

$$
    \mathbf{s} =
    \begin{pmatrix}
        0 \\
        0 \\
        0 \\
        0 \\
        0 \\
        0
    \end{pmatrix}
$$

We can convert this into an augmented skew symmetric matrix

$$
    [\mathbf{s}] =
    \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0
    \end{pmatrix}
$$

and we can convert this matrix into an $\mathbf{SE}(3)$ matrix using the matrix exponential

\begin{align*}
    \mathbf{T}(\mathbf{s})
    &= e^{[\mathbf{s}]} \\
    &=
    \begin{pmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1
    \end{pmatrix} 
\end{align*}

which gives us the identity matrix.


## Rotation only Twist

What happens if we make a twist around the $z$-axis and no translation component?

$$
    \mathbf{s}(\theta) =
    \begin{pmatrix}
        0 \\
        0 \\
        0 \\
        0 \\
        0 \\
        \theta
    \end{pmatrix}
$$

We can convert this into an augmented skew symmetric matrix

$$
    [\mathbf{s}(\theta)] =
    \begin{pmatrix}
        0 & -\theta & 0 & 0 \\
        \theta & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0
    \end{pmatrix}
$$

and we can convert this matrix into an $\mathbf{SE}(3)$ matrix using the matrix exponential

\begin{align*}
    \mathbf{T}(\theta)
    &= e^{[\mathbf{s}(\theta)]} \\
    &=
    \begin{pmatrix}
        \cos(\theta) & -\sin(\theta) & 0 & 0 \\
        \sin(\theta) & \cos(\theta) & 0 & 0 \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1
    \end{pmatrix}
\end{align*}



And we can see this is just a rotation matrix around the $z$-axis embedded in an $\mathbf{SE}(3)$ matrix.

## Translation only Twist

What happens if we make a twist with no rotation component and a translation component?

$$
    \mathbf{s}(\upsilon) =
    \begin{pmatrix}
        \upsilon_1 \\
        \upsilon_2 \\
        \upsilon_3 \\
        0 \\
        0 \\
        0
    \end{pmatrix}
$$

We can convert this into an augmented skew symmetric matrix

$$
    [\mathbf{s}(\upsilon)] =
    \begin{pmatrix}
        0 & 0 & 0 & \upsilon_1 \\
        0 & 0 & 0 & \upsilon_2 \\
        0 & 0 & 0 & \upsilon_3 \\
        0 & 0 & 0 & 0
    \end{pmatrix}
$$

and we can convert this matrix into an $\mathbf{SE}(3)$ matrix using the matrix exponential

\begin{align*}
    \mathbf{T}(\upsilon)
    &= e^{[\mathbf{s}(\upsilon)]} \\
    &=
    \begin{pmatrix}
        1 & 0 & 0 & \upsilon_1 \\
        0 & 1 & 0 & \upsilon_2 \\
        0 & 0 & 1 & \upsilon_3 \\
        0 & 0 & 0 & 1
    \end{pmatrix}
\end{align*}



For the case where there is no rotation, the translation component of the twist $\mathbf{\upsilon}$ to the translation component $\mathbf{t}$ of the $\mathbf{SE}(3)$ matrix.

In [3]:
x = sym.Symbol('x')
y = sym.Symbol('y')
z = sym.Symbol('\\theta')

# s = np.array([
#     [0, -z, x],
#     [z, 0, y],
#     [0, 0, 0]
# ])

# t = sym.simplify(sym.Matrix(s).exp())

# ds.display(ds.Math(f"exp(s) = {sym.latex(sym.simplify(t))}"))


theta = 1.0
s = np.array([
    [0, -theta, 1.0],
    [theta, 0, 2.0],
    [0, 0, 0]
])

t = expm(s)

# ds.display(ds.Math(f"exp(s) = {sym.latex(sym.simplify(t))}"))

print(np.round(t, 3))

print(np.linalg.norm(t[:2, -1]))
print(np.linalg.norm(s[:2, -1]))

[[ 0.54  -0.841 -0.078]
 [ 0.841  0.54   2.143]
 [ 0.     0.     1.   ]]
2.144056188936895
2.23606797749979


In [4]:
a = sym.Symbol('a')

g = np.array([[0, -1], [1, 0]])

g_theta = sym.Matrix(a * g)

ds.display(ds.Math(f"g_{{\\theta}} = {sym.latex(g_theta)}"))

r = sym.simplify(sym.Matrix(a * g).exp())

ds.display(ds.Math(f"R = {sym.latex(sym.Matrix(r))}"))

dr = sym.diff(r, a)

ds.display(ds.Math(f"dR = {sym.latex(sym.Matrix(dr))}"))

theta = np.pi/4

dr_eye = sym.simplify(dr.subs(a, theta)).evalf()

ds.display(ds.Math(f"dR(\\theta={theta}) = {sym.latex(sym.Matrix(dr_eye))}"))

r_eval = sym.simplify(r.subs(a, theta)).evalf()

ds.display(ds.Math(f"R(\\theta={theta}) = {sym.latex(sym.Matrix(r_eval))}"))

what = dr_eye @ r_eval.T

ds.display(ds.Math(f"{{\\omega}} = {sym.latex(sym.Matrix(what))}"))

ds.display(ds.Math(f"\\eta = {sym.latex(sym.Matrix(np.round(logm(np.array(r_eval, dtype=np.float64)), 2)))}"))


# rs = sym.Matrix(sm.SO2(a).A)

# print(rs)

# drs = sym.diff(rs, a)

# print(drs)



# print(drs.subs(a, 1.5))



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:
s = sm.Twist3([1, 2, 4, 0, 0, 30])

print(s)

print(s.exp())

print(s.pole)


(1 2 4; 0 0 30)
   0.1543    0.988     0        -0.08932   
  -0.988     0.1543    0        -0.03768   
   0         0         1         4         
   0         0         0         1         

[-2.  1.  0.]


In [6]:
x, y, z = sym.symbols('x y z')

t = np.array([x, y, z])

a = np.array([0.041, 0.408, 0.789])
theta = np.linalg.norm(a)
ahat = a/theta

print(ahat)

print(theta)

askew = smb.skew(ahat)

print(askew @ t)

[0.04610917 0.45884249 0.88732041]
0.8891940170738892
[-0.887320410225429*y + 0.458842493500602*z
 0.887320410225429*x - 0.0461091721409918*z
 -0.458842493500602*x + 0.0461091721409918*y]


<br>

<a id='2'></a>
# 2.0 Interpolation of Orientation
---

In robotics, we often need to interpolate orientation. This essentially means that we wish to smoothly transition from one orientation to another and to achieve this we create a sequence of orientations that smoothly approach the desired orientation while starting at the original.

More specifically, given two orientations ${^0\mathbf{R}_a}$ and ${^a\mathbf{R}_b}$, we require a function $f(s) = \sigma({^0\mathbf{R}_a}, {^0\mathbf{R}_b}, s)$ where $s \in [0, 1]$ and we have boundary conditions that give $f(0) = {^0\mathbf{R}_a}$ and $f(1) = {^0\mathbf{R}_b}$. The function must vary _smoothly_ for intermediate values of $s$.

We wish to interpolate in the context of __robotics__. This means that we have boundary conditions. A drone, car, or manipulator cannot instantly _step_ in velocity, to do so would require infinite acceleration. The boundary conditions are that at time $t=0$ we have a set velocity, acceleration, jerk, and so on. We also have a desired velocity, acceleration, jerk, and so on at time $t=T$. Our interpolation function must satisfy these boundary conditions and also smoothly transition between the two for each of these values.

## 2.1 Smooth One-Dimensional Trajectory

We start our discussion with a scalar function of time that has a
specified initial and final value.
An obvious candidate for such a smooth function is a polynomial
function of time. Polynomials are
simple to compute and can easily provide the required smoothness and boundary conditions. A quintic
(fifth-order) polynomial is commonly used

$$
	q(t)=At^{5}+Bt^{4}+Ct^{3}+Dt^{2}+Et+F
$$

where time $t\in [0,T]$. The first- and second-derivatives are also polynomials

\begin{align*}
	\dot{q}(t)  &= 5At^{4}+4Bt^{3}+3Ct^{2}+2Dt+E \\
	\ddot{q}(t) &= 20At^{3}+12Bt^{2}+6Ct+2D
\end{align*}

and therefore smooth. The third-derivative, jerk, will be a quadratic.

The trajectory has defined boundary conditions for position ($q_0$, $q_T$), velocity ($\dot{q}_0$, $\dot{q}_T$) and
acceleration ($\ddot{q}_0$, $\ddot{q}_T$).
Writing $q(t)$ to $\ddot{q}(t)$ for the boundary conditions $t=0$ and $t=T$ gives six
equations which we can write in matrix form as

$$
	\left(\begin{array}{c}
		q_{0}\cr
		q_{T}\cr
		\dot{q}_{0}\cr
		\dot{q}_{T}\cr
		\ddot{q}_{0}\cr
		\ddot{q}_{T}
	\end{array}\right)%þ
	=\begin{pmatrix}
		0       & 0       & 0      & 0     & 0 & 1\cr
		T^{5}   & T^{4}   & T^{3}  & T^{2} & T & 1\cr
		0       & 0       & 0      & 0     & 1 & 0\cr
		5T^{4}  & 4T^{3}  & 3T^{2} & 2T    & 1 & 0\cr
		0       & 0       & 0      & 2     & 0 & 0\cr
		20T^{3} & 12T^{2} & 6T     & 2     & 0 & 0
	\end{pmatrix}
	\left(\begin{array}{c}
		A\cr
		B\cr
		C\cr
		D\cr
		E\cr
		F
	\end{array}\right)  \,\,.
$$

The matrix is square and, if $T \ne 0$, we can invert it to solve for the coefficient vector $(A, B, C, D, E, F)$.

_This is the reason for choice of quintic polynomial. It has six coefficients that enable it to meet the six boundary conditions on initial and final position, velocity and acceleration._

In [3]:
q0 = 0
qf = 1.0

T = 10.0

qd0 = 0.0
qdf = 0.0

qdd0 = 0.0
qddf = 0.0

q = np.array([
    [q0],
    [qf],
    [qd0],
    [qdf],
    [qdd0],
    [qddf],
])

X = np.array([
    [0, 0, 0, 0, 0, 1],
    [T**5, T**4, T**3, T**2, T, 1],
    [0, 0, 0, 0, 1, 0],
    [5 * T**4, 4 * T**3, 3 * T**2, 2 * T, 1, 0],
    [0, 0, 0, 2, 0, 0],
    [20 * T**3, 12 * T**2, 6 * T, 2, 0, 0],
])

coeffs = np.linalg.inv(X) @ q

## Interpolation using RPY Angles

A simple approach to interpolation is to convert or rotations to roll-pitch-yaw angles, $\Gamma \in (\mathbf{S^1})^3$ and use linear interpolation

$$
    \sigma(\Gamma_0, \ \Gamma_1, \ s) = (1 - s) \Gamma_0 + s \Gamma_1
$$

where $s \in [0, 1]$. We can then convert the interpolated roll-pitch-yaw angles back to a rotation matrix.

In [37]:
R0 = sm.SO3.Rz(-1) * sm.SO3.Ry(-1)
R1 = sm.SO3.Rz(1) * sm.SO3.Ry(1)

rpy0 = np.expand_dims(R0.rpy(), axis=1)
rpy1 = np.expand_dims(R1.rpy(), axis=1)

ds.display(ds.Math(f"R_0 = {sym.latex(sym.Matrix(np.round(R0.A, 2)))}"))
ds.display(ds.Math(f"R_1 = {sym.latex(sym.Matrix(np.round(R1.A, 2)))}"))

traj1 = rtb.mtraj(rtb.quintic, rpy0, rpy1, 50)

s = np.linspace(0, 1, 50)

traj2 = (1 - s) * rpy0 + s * rpy1

print(traj1.q[:5, :].T)

print(traj2[:, :5])


<IPython.core.display.Math object>

<IPython.core.display.Math object>

[[ 0.          0.          0.          0.          0.        ]
 [-1.         -0.99983516 -0.99872193 -0.99582128 -0.9904089 ]
 [-1.         -0.99983516 -0.99872193 -0.99582128 -0.9904089 ]]
[[ 0.          0.          0.          0.          0.        ]
 [-1.         -0.95918367 -0.91836735 -0.87755102 -0.83673469]
 [-1.         -0.95918367 -0.91836735 -0.87755102 -0.83673469]]
